In [1]:
# ==============================
# Step 4: Build & Train Model
# ==============================

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# ------------------------------
# 1️⃣ Paths
# ------------------------------
PROCESSED_DIR = r"C:\Users\uthay\Desktop\cv-waste-classification\data\processed"
TRAIN_DIR = os.path.join(PROCESSED_DIR, "train")
VAL_DIR = os.path.join(PROCESSED_DIR, "val")
TEST_DIR = os.path.join(PROCESSED_DIR, "test")

# ------------------------------
# 2️⃣ Hyperparameters
# ------------------------------
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10  # you can increase later
NUM_CLASSES = 10
LEARNING_RATE = 1e-4

# ------------------------------
# 3️⃣ ImageDataGenerators
# ------------------------------
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_test_gen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = val_test_gen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# ------------------------------
# 4️⃣ Load Pretrained Model
# ------------------------------
base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model
base_model.trainable = False

# ------------------------------
# 5️⃣ Add Custom Layers
# ------------------------------
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# ------------------------------
# 6️⃣ Compile Model
# ------------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ------------------------------
# 7️⃣ Callbacks
# ------------------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('models/best_model.h5', monitor='val_loss', save_best_only=True)
]

# ------------------------------
# 8️⃣ Train Model
# ------------------------------
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=callbacks
)


C:\Users\uthay\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Found 14142 images belonging to 10 classes.
Found 3027 images belonging to 10 classes.
Found 3043 images belonging to 10 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 18s 2us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,270,794 (8.66 MB)

...

442/442 ━━━━━━━━━━━━━━━━━━━━ 806s 2s/step - accuracy: 0.8454 - loss: 0.4754 - val_accuracy: 0.8867 - val_loss: 0.3771
Epoch 7/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8491 - loss: 0.4625  

442/442 ━━━━━━━━━━━━━━━━━━━━ 768s 2s/step - accuracy: 0.8531 - loss: 0.4547 - val_accuracy: 0.8933 - val_loss: 0.3582
Epoch 8/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8614 - loss: 0.4369  

442/442 ━━━━━━━━━━━━━━━━━━━━ 734s 2s/step - accuracy: 0.8621 - loss: 0.4301 - val_accuracy: 0.8949 - val_loss: 0.3477
Epoch 9/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8721 - loss: 0.4045   

442/442 ━━━━━━━━━━━━━━━━━━━━ 726s 2s/step - accuracy: 0.8699 - loss: 0.4047 - val_accuracy: 0.8979 - val_loss: 0.3380
Epoch 10/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8717 - loss: 0.3874   

442/442 ━━━━━━━━━━━━━━━━━━━━ 724s 2s/step - accuracy: 0.8679 - loss: 0.3990 - val_accuracy: 0.9042 - val_loss: 0.3254


In [3]:
import os

# create models folder
os.makedirs("models", exist_ok=True)

# save the trained model NOW
model.save("models/best_model.keras")

print("✅ Model saved without retraining")


✅ Model saved without retraining
